# Visualizations
## Preparing Data for Kepler.gl

# What are we looking to accomplish?

Here we will transform our speed and delays data into a format that can be consumed by the **kepler.gl** tool. Data that is deemed important to include for visualization or filtering is all included and merged.

In [1]:
import pandas as pd
import re
import numpy as np
from keplergl import KeplerGl

In [2]:
# Cleaning the stops from letters and zeros
stops = pd.read_csv('../data/raw/gtfs/stops.txt')
stops = stops.drop(columns=['stop_code', 'stop_desc','zone_id','stop_url', 'location_type', 'parent_station'])
stops = stops # remove the .copy() if you want the change to be made in stops, or do it directly there
stops['stop_id'] = stops['stop_id'].map(lambda x: int(re.sub('[^0-9]', '', x))) # clean the letters and zeros
stops = stops.drop_duplicates(subset=['stop_id'], keep='first') # remove the duplicates
len(stops)

2842

In [3]:
line_stops = pd.read_csv('../data/processed/assignment1/line_stops.csv')
# Extracting the information we need from line_stops
line_stops = line_stops[['lineId','direction', 'name_ascii', 'stop_id_int','order', 'lat', 'long']]
line_stops['type'] = line_stops['lineId'].apply(lambda x: str(x[-1])) # getting the type m/b/t 
line_stops['lineId'] = line_stops['lineId'].apply(lambda x: int(x[:-1])) # lineId number
line_stops = line_stops.drop(line_stops[line_stops.lineId >= 100].index) # dropping the night buses
line_stops = line_stops.drop_duplicates(subset=['lineId', 'direction', 'stop_id_int','order'])
len(line_stops)

3596

# Speed - Assignment 1


In [24]:
vehicle_speed_path = '../data/processed/assignment1/vehicleSpeedReformatted.csv'
vehicle_speed = pd.read_csv(vehicle_speed_path)
vehicle_speed.head(1)

,LineId,FromStop,ToStop,Day,LineIdFormatted,LineId_GeoMerge,Type,Direction,fromIndex,toIndex,FromStop_lat,FromStop_lon,ToStop_lat,ToStop_lon,geojson,hour,speed
0,37,2957,5810,NaN,line37,037b-1,b,1.0,49.0,57.0,50.821413,4.341859,50.818572,4.340952,"{'type': 'FeatureCollection', 'features': [{'t...",00:00,14.936818


In [25]:
# vehicle_speed = vehicle_speed[['LineId', 'Type', 'FromStop', 'ToStop', 'Direction', 'hour', 'speed', 'geojson']]
vehicle_speed = vehicle_speed[['LineId', 'FromStop', 'ToStop', 'Direction', 'hour', 'speed', 'geojson','FromStop_lat','FromStop_lon','ToStop_lat','ToStop_lon']]

# Remove the inf values - we drop like 20k values
vehicle_speed = vehicle_speed.replace([np.inf], np.nan)
vehicle_speed = vehicle_speed.dropna() 

vehicle_speed = vehicle_speed.astype({'Direction': int})
vehicle_speed.head(2)

,LineId,FromStop,ToStop,Direction,hour,speed,geojson,FromStop_lat,FromStop_lon,ToStop_lat,ToStop_lon
0,37,2957,5810,1,00:00,14.936818,"{'type': 'FeatureCollection', 'features': [{'t...",50.821413,4.341859,50.818572,4.340952
5,37,2957,5810,1,05:00,2.299439,"{'type': 'FeatureCollection', 'features': [{'t...",50.821413,4.341859,50.818572,4.340952


# Delays - Assignment 2

In [27]:
# Using the delays and creating the Time coulmn from the Hour column: 7 -> '07:00:00'
delays_path = '../data/processed/assignment2/final_filtered_delays.csv'

vehicle_delays = pd.read_csv(delays_path)
vehicle_delays['Time'] = vehicle_delays['Hour'].apply(lambda x: str(x.split(':')[0]+':00'))

# vehicle_delays = vehicle_delays[['LineId', 'CurrentStop', 'Date', 'Time', 'Delay', 'Day', 'Hour']]
vehicle_delays.head(3)

,Timestamp,LineId,TerminusStop,CurrentStop,Date,Hour,Day,Delay,ExpectedArrivalTime,Time
0,1630915041551,2,8472,8382,20210906,07:57:21,0,129,07:55:12,07:00
1,1630915074628,2,8472,8763,20210906,07:57:54,0,49,07:57:05,07:00
2,1630915106373,2,8472,8372,20210906,07:58:26,0,114,07:56:32,07:00


In [28]:
# joining on lineId and stopid because we can have one station on different lines 
vehicle_delays = vehicle_delays.merge(line_stops, how='inner', left_on=['LineId','CurrentStop'], right_on=['lineId','stop_id_int'] )

In [29]:
vehicle_delays = vehicle_delays[['LineId', 'CurrentStop', 'Date','Hour', 'Time', 'Delay','direction', 'Day', 'type', 'name_ascii']]

In [30]:
# cleaninug up the visual_data
vehicle_delays = vehicle_delays.drop_duplicates(subset=['LineId', 'CurrentStop', 'direction', 'Time','Day', 'type'])
# visual_data.rename(columns={'direction': 'Direction', 'order': 'Order', 'type': 'Type'}, inplace=True)
# visual_data = visual_data.astype({'LineId': str, 'Delay': 'int', 'Direction': int, 'Order': int})

In [31]:
#TODO Should we add order column?
#grouping by to get the result
grouped_delays = vehicle_delays.groupby(['LineId', 'type', 'CurrentStop', 'direction',  'Day', 'Time'], as_index=False)['Delay'].mean()
grouped_delays.head()

,LineId,type,CurrentStop,direction,Day,Time,Delay
0,2,m,8301,2,0,05:00,145.0
1,2,m,8301,2,0,06:00,113.0
2,2,m,8301,2,0,07:00,47.0
3,2,m,8301,2,0,08:00,19.0
4,2,m,8301,2,0,09:00,80.0


# The merge

Once we transformed the data for the delays and vehicle speed we can merge them in one file having LineId', 'CurrentStop', 'direction', 'Time' as mutual columns on which we will join

In [34]:
# Merging on FromStop

In [16]:
visualization_data = grouped_delays.merge(vehicle_speed, how='inner', left_on=['LineId', 'CurrentStop', 'direction', 'Time'], right_on=['LineId','FromStop','Direction','hour'])
visualization_data = visualization_data.drop(columns=['direction'])

In [17]:
visualization_data.head(1)

,LineId,type,CurrentStop,Day,Time,Delay,FromStop,ToStop,Direction,hour,speed,geojson,FromStop_lat,FromStop_lon,ToStop_lat,ToStop_lon
0,1,m,8011,0,05:00,58.0,8011,8271,2,05:00,11.219475,"{'type': 'FeatureCollection', 'features': [{'t...",50.850095,4.352165,50.852347,4.347749


In [18]:
# adding the 'line'+id to the LineId column so that we use it as string in the visualization
visualization_data['LineId'] = visualization_data['LineId'].apply(lambda x: 'line'+str(x)) 

In [19]:
visualization_data_stops = visualization_data.merge(line_stops[['name_ascii','stop_id_int']], how='inner', left_on='CurrentStop', right_on='stop_id_int').drop(columns=['stop_id_int'])

In [20]:
visualization_data = visualization_data_stops.drop_duplicates()

We can drop the geojson column or we can use it in our data. Note that with the geojson we will generate a table of 2.5GB which kepler has difficulties runing. 

In [21]:
visualization_data_no_geojson = visualization_data.drop(columns=['geojson'])

In [22]:
visualization_data_no_geojson.to_csv('../data/processed/Visualization/visualization_data.csv')

In [23]:
visualization_data_no_geojson

,LineId,type,CurrentStop,Day,Time,Delay,FromStop,ToStop,Direction,hour,speed,FromStop_lat,FromStop_lon,ToStop_lat,ToStop_lon,name_ascii
0,line1,m,8011,0,05:00,58.0,8011,8271,2,05:00,11.219475,50.850095,4.352165,50.852347,4.347749,DE BROUCKERE
2,line1,m,8011,1,05:00,-198.0,8011,8271,2,05:00,11.219475,50.850095,4.352165,50.852347,4.347749,DE BROUCKERE
4,line1,m,8011,0,06:00,66.0,8011,8271,2,06:00,11.554020,50.850095,4.352165,50.852347,4.347749,DE BROUCKERE
6,line1,m,8011,1,06:00,71.0,8011,8271,2,06:00,11.554020,50.850095,4.352165,50.852347,4.347749,DE BROUCKERE
8,line1,m,8011,0,07:00,-47.0,8011,8271,2,07:00,10.736122,50.850095,4.352165,50.852347,4.347749,DE BROUCKERE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224958,line98,b,9686,0,17:00,-55.0,9686,5964,2,17:00,30.144700,50.794407,4.302507,50.803967,4.302302,DORENT
224959,line98,b,9686,1,17:00,110.0,9686,5964,2,17:00,30.144700,50.794407,4.302507,50.803967,4.302302,DORENT
224960,line98,b,9686,0,18:00,109.0,9686,5964,2,18:00,30.133882,50.794407,4.302507,50.803967,4.302302,DORENT
224961,line98,b,9686,1,18:00,314.0,9686,5964,2,18:00,30.133882,50.794407,4.302507,50.803967,4.302302,DORENT


# Visuals

In [43]:
visualization_data_no_geojson = pd.read_csv('../data/processed/Visualization/visualization_data.csv')


In [38]:
map_1 = KeplerGl(height=600, data={"data": visualization_data_no_geojson, "config": config})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [39]:
map_1

KeplerGl(data={'data':         LineId type  CurrentStop  Day   Time  Delay  FromStop  ToStop  \
0        line1…